# Importing BGL Dataset to s3

> Alternative: Check out importing to EFS

> Note: I have already downloaded this file to S3 ---> But that was with the earlier Notebook Instance application of SageMaker
>
> I am now teseting this download again --> fyi only

In [10]:
import os
print(os.environ)

environ({'REGION_NAME': 'us-east-1', 'HOSTNAME': 'sagemaker-data-scien-ml-m5-2xlarge-58ec53cbfb4afb44281d61bdec8c', 'SAGEMAKER_SPACE_TYPE_LOWERCASE': '', 'HOME': '/root', 'AWS_CONTAINER_CREDENTIALS_RELATIVE_URI': '/_sagemaker-instance-credentials/b817998ff9dfad5b4d5053dc6478e169d5c7fd476140ab9d7be1908a6a96f3cb', 'PYTHONNOUSERSITE': '0', 'AWS_DEFAULT_REGION': 'us-east-1', 'SAGEMAKER_SPACE_NAME': 'w210-capstone-shared', 'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/amazon/bin:/tmp/miniconda3/condabin:/tmp/anaconda3/condabin:/tmp/miniconda2/condabin:/tmp/anaconda2/condabin:/tmp/mambaforge/condabin', 'AWS_ACCOUNT_ID': '110894606152', 'DEBIAN_FRONTEND': 'noninteractive', 'SHELL': '/bin/bash', 'AWS_REGION': 'us-east-1', 'AWS_INTERNAL_IMAGE_OWNER': 'Studio', 'SAGEMAKER_APP_TYPE': 'KernelGateway', 'CONDA_DIR': '/opt/.sagemakerinternal/conda', 'PWD': '/root', 'AWS_SAGEMAKER_PYTHONNOUSERSITE': '0', 'SAGEMAKER_LOG_FILE': '/var/log/studio/kernel_gateway.

In [13]:
import os
print(os.getenv('USER'))  # Should print the current user
print(os.geteuid())       # Should print the user ID, 0 for root


None
0


In [14]:
import os

# Determine the base directory based on the effective user ID
if os.geteuid() == 0:  # root user
    base_dir = '/root/11.Data/01.BGL/01.Raw_CFDR'
else:
    base_dir = '/home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR'

# Set the environment variable
os.environ['BASE_DIR'] = base_dir

# Verify the base directory
print(f"Base directory set to: {base_dir}")


Base directory set to: /root/11.Data/01.BGL/01.Raw_CFDR


In [12]:
import os

# Verify the current working directory
print("Current working directory:", os.getcwd())

# Get the base directory from the environment variable
base_dir = os.getenv('BASE_DIR')
print("Base directory:", base_dir)

# Construct and verify the file path
file_path = os.path.join(base_dir, '11.Data/01.BGL/01.Raw_CFDR/bgl2.gz')
print("File path:", file_path)

# Check if the file exists
print("File exists:", os.path.exists(file_path))


Current working directory: /root/10.EDA
Base directory: /home/sagemaker-user
File path: /home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz
File exists: False


In [15]:
import os

# Verify the current working directory
print("Current working directory:", os.getcwd())

# Get the base directory from the environment variable
base_dir = os.getenv('BASE_DIR')
print("Base directory:", base_dir)

# Construct and verify the file path
file_path = os.path.join(base_dir, 'bgl2.gz')
print("File path:", file_path)

# Check if the file exists
print("File exists:", os.path.exists(file_path))


Current working directory: /root/10.EDA
Base directory: /root/11.Data/01.BGL/01.Raw_CFDR
File path: /root/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz
File exists: True


### Import to S3

In [2]:
# import boto3
# import requests
# import os

# # Define the URL and local file path
# url = 'http://0b4af6cdc2f0c5998459-c0245c5c937c5dedcca3f1764ecc9b2f.r43.cf2.rackcdn.com/hpc4/bgl2.gz'
# local_filename = 'bgl2.gz'

# # S3 bucket details
# s3_bucket = 'w210-capstone'
# s3_key = '11.Data/20240524_BlueGene_Raw_HPC/Temp_DNU/bgl2.gz'

# ######### Location of Actual File ########
# # 11.Data/20240524_BlueGene_Raw_HPC/bgl2.gz

# # Download the file from `url` and save it locally
# with requests.get(url, stream=True) as r:
#     r.raise_for_status()
#     with open(local_filename, 'wb') as f:
#         for chunk in r.iter_content(chunk_size=8192):
#             if chunk:  # filter out keep-alive new chunks
#                 f.write(chunk)

# print(f"Downloaded {local_filename} successfully.")

# # Upload the file to the specified S3 location
# s3 = boto3.client('s3')
# try:
#     s3.upload_file(local_filename, s3_bucket, s3_key)
#     print(f"Uploaded {local_filename} to s3://{s3_bucket}/{s3_key} successfully.")
# except Exception as e:
#     print(f"An error occurred while uploading to S3: {e}")

# # Clean up the local file
# os.remove(local_filename)
# print(f"Cleaned up local file {local_filename}.")


### Download Directory to local EFS Folder

In [3]:
# import os
# import requests

# # Change to the root directory
# os.chdir('/root')

# # Define the URL and the local file path
# url = 'http://0b4af6cdc2f0c5998459-c0245c5c937c5dedcca3f1764ecc9b2f.r43.cf2.rackcdn.com/hpc4/bgl2.gz'
# # local_dir = '/home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR'
# local_dir = '11.Data/01.BGL/01.Raw_CFDR'

# local_filename = 'bgl2.gz'
# local_filepath = os.path.join(local_dir, local_filename)

# # Create the directory if it doesn't exist
# os.makedirs(local_dir, exist_ok=True)

# # Download the file from `url` and save it locally under `local_filepath`
# with requests.get(url, stream=True) as r:
#     r.raise_for_status()
#     with open(local_filepath, 'wb') as f:
#         for chunk in r.iter_content(chunk_size=8192):
#             f.write(chunk)

# print(f"Downloaded {local_filepath}")


---

In [5]:
import os
import gzip
import pandas as pd
import matplotlib.pyplot as plt

# Get the base directory from the environment variable
# base_dir = os.getenv('BASE_DIR', '/home/sagemaker-user')  # Default to /home/sagemaker-user if not set

# Define the file path
# file_path = os.path.join(base_dir, '11.Data/01.BGL/01.Raw_CFDR/bgl2.gz')
file_path = '/root/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'

# Load and read the data
with gzip.open(file_path, 'rt') as f:
    # Read the file into a DataFrame
    lines = f.readlines()

# Print the first few lines to understand the structure
for line in lines[:10]:
    print(line.strip())

- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.131467 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.293532 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instr

### <font color = grey> Total Counts

In [7]:
import gzip

# Define the file path
# file_path = '/home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'
file_path = '/root/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'

# Count the number of lines
line_count = 0
with gzip.open(file_path, 'rt') as f:
    for line in f:
        line_count += 1

print(f'Total number of rows: {line_count}')

Total number of rows: 4747963


---

### <font color = grey> Line Position Analysis

In [8]:
import gzip

# Define the file path
# file_path = '/home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'
file_path = '/root/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'

# Initialize counters
total_lines = 0
lines_starting_with_hyphen = 0
lines_not_starting_with_hyphen = 0

# Count the number of lines and categorize them
with gzip.open(file_path, 'rt') as f:
    for line in f:
        total_lines += 1
        if line.startswith('-'):
            lines_starting_with_hyphen += 1
        else:
            lines_not_starting_with_hyphen += 1

# Print the results
print(f'Total number of rows: {total_lines}')
print(f'Number of rows starting with "-": {lines_starting_with_hyphen}')
print(f'Number of rows not starting with "-": {lines_not_starting_with_hyphen}')

Total number of rows: 4747963
Number of rows starting with "-": 4399265
Number of rows not starting with "-": 348698


### <font color = grey> first 10 rows of exceptions

In [9]:
import gzip

# Define the file path
# file_path = '/home/sagemaker-user/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'
file_path = '/root/11.Data/01.BGL/01.Raw_CFDR/bgl2.gz'

# Initialize counters and storage for exception lines
total_lines = 0
lines_starting_with_hyphen = 0
lines_not_starting_with_hyphen = 0
exception_lines = []

# Count the number of lines and categorize them
with gzip.open(file_path, 'rt') as f:
    for line in f:
        total_lines += 1
        if line.startswith('-'):
            lines_starting_with_hyphen += 1
        else:
            lines_not_starting_with_hyphen += 1
            if len(exception_lines) < 10:
                exception_lines.append(line.strip())

# Print the results
print(f'Total number of rows: {total_lines}')
print(f'Number of rows starting with "-": {lines_starting_with_hyphen}')
print(f'Number of rows not starting with "-": {lines_not_starting_with_hyphen}')

# Print the first 10 exception lines
print("\nFirst 10 lines that do not start with '-':")
for i, exception_line in enumerate(exception_lines, 1):
    print(f"{i}: {exception_line}")


Total number of rows: 4747963
Number of rows starting with "-": 4399265
Number of rows not starting with "-": 348698

First 10 lines that do not start with '-':
1: R_DDR_EXC 1117840321 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.01.215908 R16-M1-N2-C:J17-U01 RAS KERNEL INFO ddr: excessive soft failures, consider replacing the card
2: R_DDR_EXC 1117846777 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-17.59.37.564438 R16-M1-N2-C:J17-U01 RAS KERNEL INFO ddr: excessive soft failures, consider replacing the card
3: APPREAD 1117869872 2005.06.04 R23-M1-N8-I:J18-U11 2005-06-04-00.24.32.398284 R23-M1-N8-I:J18-U11 RAS APP FATAL ciod: failed to read message prefix on control stream (CioStream socket to 172.16.96.116:33399
4: APPREAD 1117869872 2005.06.04 R04-M1-N4-I:J18-U11 2005-06-04-00.24.32.432192 R04-M1-N4-I:J18-U11 RAS APP FATAL ciod: failed to read message prefix on control stream (CioStream socket to 172.16.96.116:33569
5: APPREAD 1117869872 2005.06.04 R04-M1-N4-I:J18-U01 2005-06-04-00.24